# This notebook shows the second part of the interpolation experiments

In [ ]:
import numpy as np
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from tqdm.auto import tqdm, trange

In [ ]:
import plotly.io as pio

def plot_signal(x, t):
    return go.Figure([go.Scatter(y=x, x=t, mode="lines+markers")
                      ]).update_layout(
        xaxis_title="Time, s",
        yaxis_title="Signal",
    )

def plot_freqs(x, f, f_max=100):
    f_idxs = np.abs(f)<f_max
    return go.Figure(go.Scatter(y=x[f_idxs], x=f[f_idxs], mode="lines+markers")).update_layout(
        xaxis_title="Frequency, Hz",
        yaxis_title="Strength",
    )

def fft(vals, ts):
    sample_rate = ts.shape[-1]/(ts[-1] - ts[0])
    return np.abs(np.fft.fft(vals)), np.fft.fftfreq(vals.shape[-1])*sample_rate


In [ ]:
def wsinterp(x, xp, fp, left=None, right=None):
    """One-dimensional Whittaker-Shannon interpolation.

    This uses the Whittaker-Shannon interpolation formula to interpolate the
    value of fp (array), which is defined over xp (array), at x (array or
    float).

    Returns the interpolated array with dimensions of x.

    x: t_int
    xp: t_real
    fp: f_real

    """
    scalar = np.isscalar(x)
    if scalar:
        x = np.array(x)
        x.resize(1)
    Tn = (xp[-1] - xp[0])/xp.shape[0]

    # shape = (nxp, nx), nxp copies of x data span axis 1
    u = np.resize(x, (len(xp), len(x)))
    # Must take transpose of u for proper broadcasting with xp.
    # shape = (nx, nxp), v(xp) data spans axis 1
    v = (xp - u.T) / (Tn)
    # shape = (nx, nxp), m(v) data spans axis 1
    m = fp * np.sinc(v)
    # Sum over m(v) (axis 1)
    fp_at_x = np.sum(m, axis=1)

    # Enforce left and right
    if left is None:
        left = fp[0]
    fp_at_x[x < xp[0]] = left
    if right is None:
        right = fp[-1]
    fp_at_x[x > xp[-1]] = right

    # Return a float if we got a float
    if scalar:
        return float(fp_at_x)

    return fp_at_x

In [ ]:
sample_rate_full = 1000  #  Samples per s
sin_ts = np.arange(0, 3, 1/sample_rate_full)
sin_freq = 5  # Hz
sin_omega = (2*np.pi)*sin_freq
sin_freq2 = 6
# sin_vals = np.sin(sin_omega * sin_ts)

sin_vals1 = np.exp(1j * sin_omega * sin_ts)
sin_vals2 = np.exp(1j*2*np.pi*sin_freq2 * sin_ts)
sin_vals = sin_vals1+sin_vals2*0.3
plot_signal(sin_vals.real, sin_ts).show()
plot_freqs(*fft(sin_vals, sin_ts)).show()

In [ ]:
ids_irreg = np.sort(np.random.choice(np.arange(sin_ts.shape[0]), (sin_ts.shape[0])//20, False))
sin_vals_irreg = sin_vals[ids_irreg]
sin_ts_irreg = sin_ts[ids_irreg]

# plot_signal(sin_vals_irreg.real, sin_ts_irreg).show()
fig = go.Figure()
fig.add_trace(go.Scatter(y=sin_vals.real, x=sin_ts, mode="lines", name="Siganl"))
fig.add_trace(go.Scatter(y=sin_vals_irreg.real, x=sin_ts_irreg, mode="markers", name="Sample points"))
fig.show()

In [ ]:
plot_freqs(*fft(sin_vals_irreg, sin_ts_irreg)).show()

In [ ]:
sin_wsint = wsinterp(sin_ts, sin_ts_irreg, sin_vals_irreg)
# plot_signal(sin_wsint.real, sin_ts).show()

fig = go.Figure()
fig.add_trace(go.Scatter(y=sin_wsint.real, x=sin_ts, mode="lines", name="Interpolated"))
fig.add_trace(go.Scatter(y=sin_vals_irreg.real, x=sin_ts_irreg, mode="markers", name="Sample points"))
fig.show()

plot_freqs(*fft(sin_wsint, sin_ts)).show()

In [ ]:

def randomized_sinc_interp(x:np.ndarray, xp:np.ndarray, fp:np.ndarray, sigma_coeff=0.8, left=None, right=None)->np.ndarray:

    Tn = (xp[-1] - xp[0])/xp.shape[0]
#     print(xp.shape, xp[0], xp[1], Tn)
    xp_regular = np.arange(xp[0], xp[0] + Tn*xp.shape[0], Tn)
    
    xp_deltas = xp - xp_regular

    xp_result = xp_regular + xp_deltas * sigma_coeff 

    # shape = (nxp, nx), nxp copies of x data span axis 1
    u = np.resize(x, (len(xp), len(x)))
    # Must take transpose of u for proper broadcasting with xp.
    # shape = (nx, nxp), v(xp) data spans axis 1
    # v = (xp - u.T) / (Tn)
    v = (xp_result - u.T) / (Tn)
    # shape = (nx, nxp), m(v) data spans axis 1
    m =   fp * np.sinc(v)
    # Sum over m(v) (axis 1)
    fp_at_x = np.sum(m, axis=1)

    # Enforce left and right
    if left is None:
        left = fp[0]
    fp_at_x[x < xp[0]] = left
    if right is None:
        right = fp[-1]
    fp_at_x[x > xp[-1]] = right

    return fp_at_x


In [ ]:

for sigma_coeff in (0, 0.01, 0.1, 0.2, 0.4, 0.6, 0.8, 1, 1.2):
    print(sigma_coeff)
    sin_int = randomized_sinc_interp(sin_ts, sin_ts_irreg, sin_vals_irreg, sigma_coeff = sigma_coeff)
    # plot_signal(sin_wsint.real, sin_ts).show()

    fig = go.Figure()
    fig.add_trace(go.Scatter(y=sin_wsint.real, x=sin_ts, mode="lines", name="Interpolated"))
    fig.add_trace(go.Scatter(y=sin_vals_irreg.real, x=sin_ts_irreg, mode="markers", name="Sample points"))
    fig.show()

    plot_freqs(*fft(sin_int, sin_ts)).show()

In [ ]:
from scipy import interpolate

def splineinterp(x, xp, fp):
    tck = interpolate.splrep(xp, fp)
    return interpolate.splev(x, tck)

In [ ]:
sin_spint = splineinterp(sin_ts, sin_ts_irreg, sin_vals_irreg.real) + 1j*splineinterp(sin_ts, sin_ts_irreg, sin_vals_irreg.imag)
# plot_signal(sin_spint.imag, sin_ts).show()

fig = go.Figure()
fig.add_trace(go.Scatter(y=sin_spint.real, x=sin_ts, mode="lines", name="Interpolated"))
fig.add_trace(go.Scatter(y=sin_vals.real, x=sin_ts, mode="lines", name="Real"))
fig.add_trace(go.Scatter(y=sin_vals_irreg.real, x=sin_ts_irreg, mode="markers", name="Sample points"))
fig.show()

plot_freqs(*fft(sin_spint, sin_ts)).show()